In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
class InceptionV1(keras.layers.Layer):
    def __init__(self, num_filters=28, strides=1, activation="relu", padding="same"):
        super(InceptionV1, self).__init__()
        self.num_filters = num_filters
        self.strides = strides
        self.activation = activation
        self.padding = padding

    def build(self, input_shape):
        self.conv_block_a = keras.layers.Conv2D(self.num_filters, (1, 1), activation=self.activation, strides=self.strides, padding=self.padding)
        self.conv_block_b = keras.layers.Conv2D(self.num_filters, (3, 3), activation=self.activation, strides=self.strides, padding=self.padding)
        self.conv_block_c = keras.layers.Conv2D(self.num_filters, (5, 5), activation=self.activation, strides=self.strides, padding=self.padding)
        self.maxpool_block = keras.layers.MaxPool2D(pool_size=(3, 3), strides=self.strides, padding=self.padding)
    
    def call(self, x):
        out_a = self.conv_block_a(x)
        out_b = self.conv_block_b(x)
        out_c = self.conv_block_c(x)
        out_d = self.maxpool_block(x)
        output = keras.layers.concatenate([out_a, out_b, out_c, out_d], axis=-1)
        return output

In [3]:
class InceptionV1Reduction(keras.layers.Layer):
    def __init__(self, num_filters=28, strides=1, activation="relu", padding="same"):
        super(InceptionV1Reduction, self).__init__()
        self.num_filters = num_filters
        self.strides = strides
        self.activation = activation
        self.padding = padding

    def build(self, input_shape):
        self.conv_1a = keras.layers.Conv2D(self.num_filters, (1, 1), activation=self.activation, strides=self.strides*self.strides, padding=self.padding)
        self.conv_1b = keras.layers.Conv2D(self.num_filters, (1, 1), activation=self.activation, strides=self.strides, padding=self.padding)
        self.conv_1c = keras.layers.Conv2D(self.num_filters, (1, 1), activation=self.activation, strides=self.strides, padding=self.padding)
        self.conv_1d = keras.layers.Conv2D(self.num_filters, (1, 1), activation=self.activation, strides=self.strides, padding=self.padding)
        self.conv_3 = keras.layers.Conv2D(self.num_filters, (3, 3), activation=self.activation, strides=self.strides, padding=self.padding)
        self.conv_5 = keras.layers.Conv2D(self.num_filters, (5, 5), activation=self.activation, strides=self.strides, padding=self.padding)
        self.maxpool_block = keras.layers.MaxPool2D(pool_size=(3, 3), strides=self.strides, padding=self.padding)
    
    def call(self, x):
        # Block 1
        out_a = self.conv_1a(x)
        # Block 2
        out_b_inter = self.conv_1b(x)
        out_b = self.conv_3(out_b_inter)
        # Block 3
        out_c_inter = self.conv_1c(x)
        out_c = self.conv_5(out_c_inter)
        # Block 4
        out_d_inter = self.maxpool_block(x)
        out_d = self.conv_1d(out_d_inter)
        output = keras.layers.concatenate([out_a, out_b, out_c, out_d], axis=-1)
        return output

In [4]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
#reshape data to fit model
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

In [6]:
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [7]:
X_train.ndim, X_test.ndim

(4, 4)

In [8]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

In [12]:
model = Sequential()#add model layers
model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(28,28,1)))
model.add(InceptionV1())
model.add(Conv2D(32, kernel_size=3, activation="relu"))
model.add(Flatten())
model.add(Dense(10, activation="softmax"))

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
inception_v1 (InceptionV1)   (None, 26, 26, 148)       62804     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 32)        42656     
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                184330    
Total params: 290,430
Trainable params: 290,430
Non-trainable params: 0
_________________________________________________________________
